# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
get_files('data/log_data/2018/11')

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/A/A/A')
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath,lines=True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data_df=df[['song_id','title','artist_id','duration','year']]
song_data=song_data_df.values.tolist()[0]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 218.93179, 0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert,song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id','artist_latitude','artist_longitude','artist_location','artist_name']].values.tolist()[0]
artist_data


['ARD7TVE1187B99BFB1', nan, nan, 'California - LA', 'Casual']

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files('data/log_data/2018/11')
log_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [5]:
filepath2 = log_files[0]

In [6]:
df = pd.read_json(filepath2,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df_nextsong = df[df['page']=='NextSong']
df_nextsong.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [16]:
import datetime as datetime
dt = pd.to_datetime(df_nextsong['ts'],unit='ms')
t=pd.DataFrame(dt)
t['timeStamp']=df_nextsong['ts']
t['startTime']=t['ts'].apply(lambda x:x.time())
t['hour']=t['ts'].apply(lambda x:x.hour)
t['minutes']=t['ts'].apply(lambda x:x.minute)
t['seconds']=t['ts'].apply(lambda x:x.second)
t['startDay']=t['ts'].apply(lambda x:x.date())
t['startDay']=pd.to_datetime(t['startDay']).astype(str)
t['sessionId']=df_nextsong['sessionId']
t.dtypes

ts           datetime64[ns]
timeStamp             int64
startTime            object
hour                  int64
minutes               int64
seconds               int64
startDay             object
sessionId             int64
dtype: object

In [17]:
time_df = t[['sessionId','timeStamp','hour','minutes','seconds','startDay']]
for i, row in time_df.iterrows():
    arr=list(row)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [10]:
user_df = df[['userId','firstName','lastName','gender','location']]
user_df=user_df.drop_duplicates(subset='userId')
user_df.drop([21], axis=0, inplace=True)
user_df.reset_index()
user_df=user_df[['userId','firstName','lastName','gender','location']]
user_df['userId']=user_df['userId'].astype(int)
user_df

,userId,firstName,lastName,gender,location
0,91,Jayden,Bell,M,"Dallas-Fort Worth-Arlington, TX"
1,73,Jacob,Klein,M,"Tampa-St. Petersburg-Clearwater, FL"
8,4,Alivia,Terrell,F,"Parkersburg-Vienna, WV"
22,86,Aiden,Hess,M,"La Crosse-Onalaska, WI-MN"
28,24,Layla,Griffin,F,"Lake Havasu City-Kingman, AZ"
40,26,Ryan,Smith,M,"San Jose-Sunnyvale-Santa Clara, CA"
59,49,Chloe,Cuevas,F,"San Francisco-Oakland-Hayward, CA"
62,57,Katherine,Gay,F,"San Antonio-New Braunfels, TX"
67,30,Avery,Watkins,F,"San Jose-Sunnyvale-Santa Clara, CA"
78,92,Ryann,Smith,F,"Palestine, TX"


In [50]:
user_df = df[['userId','firstName','lastName','gender','location']]
user_df=user_df.drop_duplicates(subset='userId')
user_df=user_df.reset_index()
indexAge = user_df[(user_df['gender']!='M')&(user_df['gender']!='F') ].index
dummy=user_df.drop(indexAge , inplace=True)
user_df=user_df[['userId','firstName','lastName','gender','location']]
user_df['userId']=user_df['userId'].astype(int)


userId        int64
firstName    object
lastName     object
gender       object
location     object
dtype: object

In [20]:
for i, row in user_df.iterrows():
    print(list(row))

[91, 'Jayden', 'Bell', 'M', 'Dallas-Fort Worth-Arlington, TX']
[73, 'Jacob', 'Klein', 'M', 'Tampa-St. Petersburg-Clearwater, FL']
[4, 'Alivia', 'Terrell', 'F', 'Parkersburg-Vienna, WV']
[86, 'Aiden', 'Hess', 'M', 'La Crosse-Onalaska, WI-MN']
[24, 'Layla', 'Griffin', 'F', 'Lake Havasu City-Kingman, AZ']
[26, 'Ryan', 'Smith', 'M', 'San Jose-Sunnyvale-Santa Clara, CA']
[49, 'Chloe', 'Cuevas', 'F', 'San Francisco-Oakland-Hayward, CA']
[57, 'Katherine', 'Gay', 'F', 'San Antonio-New Braunfels, TX']
[30, 'Avery', 'Watkins', 'F', 'San Jose-Sunnyvale-Santa Clara, CA']
[92, 'Ryann', 'Smith', 'F', 'Palestine, TX']
[69, 'Anabelle', 'Simpson', 'F', 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD']
[74, 'Braden', 'Parker', 'M', 'Youngstown-Warren-Boardman, OH-PA']
[88, 'Mohammad', 'Rodriguez', 'M', 'Sacramento--Roseville--Arden-Arcade, CA']
[36, 'Matthew', 'Jones', 'M', 'Janesville-Beloit, WI']
[8, 'Kaylee', 'Summers', 'F', 'Phoenix-Mesa-Scottsdale, AZ']
[12, 'Austin', 'Rosales', 'M', 'New York-Newark-

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [23]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist,row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None,None
    # insert songplay record
    songplay_data=[row.userId,row.ts,songid,artistid,row.level,row.itemInSession,row.page,row.userAgent,row.length]
    #print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


IntegrityError: insert or update on table "songplays" violates foreign key constraint "songplays_ts_fkey"
DETAIL:  Key (ts)=(1543541644796) is not present in table "time".


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.